# Assignment 2: CitiBike Data Analysis

Your notebook must display
* the complete formulation of the hypothesis to be tested
* the data tables for the unreducted datasets (first few columns)
* the data tables for the reducted datasets (first few columns)
* the plots for each dataframe, with usual rules for plotting applying: visible and readable axes, title, legend, caption.

In [16]:
import numpy as np
import pandas as pd
import sys
import os

In [23]:
!ls /gws/open/Student/citibike

201307-citibike-tripdata.zip  201508-citibike-tripdata.zip
201308-citibike-tripdata.zip  201509-citibike-tripdata.zip
201309-citibike-tripdata.zip  201510-citibike-tripdata.zip
201310-citibike-tripdata.zip  201511-citibike-tripdata.zip
201311-citibike-tripdata.zip  201512-citibike-tripdata.zip
201312-citibike-tripdata.zip  201601-citibike-tripdata.zip
201401-citibike-tripdata.zip  201602-citibike-tripdata.zip
201402-citibike-tripdata.zip  201603-citibike-tripdata.zip
201403-citibike-tripdata.zip  201604-citibike-tripdata.zip
201404-citibike-tripdata.zip  201605-citibike-tripdata.zip
201405-citibike-tripdata.zip  201606-citibike-tripdata.zip
201406-citibike-tripdata.zip  201607-citibike-tripdata.zip
201407-citibike-tripdata.zip  201608-citibike-tripdata.zip
201408-citibike-tripdata.zip  201609-citibike-tripdata.zip
201409-citibike-tripdata.zip  201610-citibike-tripdata.zip
201410-citibike-tripdata.zip  201611-citibike-tripdata.zip
201411-citibike-tripdata.zip  201612-citibike-tripdata.z

In [36]:
PUIdata = os.getenv('PUIDATA')
zipfile = '/201612-citibike-tripdata.zip'
csvfile = '/201612-citibike-tripdata.csv'

if os.path.isfile(PUIdata + csvfile):
    print('CSV File found in ' + PUIdata)
else:
    if os.path.isfile(PUIdata + zipfile):
        print('ZIP file found in ' + PUIdata)
    else:
        print('File not found! Downloading ZIP file from CDF to ' + PUIdata + '...')
        os.system('cp /gws/open/Student/citibike' + zipfile + ' ' + PUIdata)
    print('Unzipping file ' + zipfile)
    os.system('unzip ' + PUIdata + zipfile + ' -d ' + PUIdata)

if os.path.isfile(PUIdata + csvfile):
    print('File is now present in ' + PUIdata)

File not found! Downloading ZIP file from CDF to /home/cusp/uc288/PUIdata...
Unzipping file /201612-citibike-tripdata.zip
File is now present in /home/cusp/uc288/PUIdata


In [37]:
data = pd.read_csv(PUIdata + csvfile)
data.head()

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender
0,528,2016-12-01 00:00:04,2016-12-01 00:08:52,499,Broadway & W 60 St,40.769155,-73.981918,228,E 48 St & 3 Ave,40.754601,-73.971879,26931,Subscriber,1964.0,1
1,218,2016-12-01 00:00:28,2016-12-01 00:04:06,3418,Plaza St West & Flatbush Ave,40.675021,-73.971115,3358,Garfield Pl & 8 Ave,40.671198,-73.974841,27122,Subscriber,1955.0,1
2,399,2016-12-01 00:00:39,2016-12-01 00:07:19,297,E 15 St & 3 Ave,40.734232,-73.986923,345,W 13 St & 6 Ave,40.736494,-73.997044,19352,Subscriber,1985.0,1
3,254,2016-12-01 00:00:44,2016-12-01 00:04:59,405,Washington St & Gansevoort St,40.739323,-74.008119,358,Christopher St & Greenwich St,40.732916,-74.007114,20015,Subscriber,1982.0,1
4,1805,2016-12-01 00:00:54,2016-12-01 00:31:00,279,Peck Slip & Front St,40.707873,-74.001670,279,Peck Slip & Front St,40.707873,-74.001670,23148,Subscriber,1989.0,1


## Idea: Women take a longer time to bike than men during weekdays. 